In [1]:
from llms_dspy.utils import get_llm, get_chroma_retriever

/home/kuba/.cache/pypoetry/virtualenvs/llms-dspy-cWHDaHg3-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import dspy


In [3]:
# This is basically from https://github.com/weaviate/recipes/blob/main/integrations/dspy/Weaviate-Import.ipynb

In [4]:
from dsp.utils import deduplicate

class GenerateAnswer(dspy.Signature):
    """Assess the context and answer the given questions that are predominantly about software usage, process optimization, and troubleshooting. Focus on providing accurate information related to tech or software-related queries."""
    
    context = dspy.InputField(desc="Helpful information for answering the question.")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="A detailed answer that is supported by the context.")

class GenerateSearchQuery(dspy.Signature):
    """Write a search query that will help answer a complex question."""

    context = dspy.InputField(desc="Contexts produced from previous search queries.")
    question = dspy.InputField(desc="The complex question we began with.")
    query = dspy.OutputField(desc="A search query that will help answer the question.")

class MultiHopRAG(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_question = dspy.ChainOfThought(GenerateSearchQuery)
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops
    
    def forward(self, question):
        context_history = []
        queries = []
        
        # Note to self, discuss how this differs from AutoGPT
        context = self.retrieve(question).passages
        for hop in range(self.max_hops):
            query = self.generate_question(context=context, question=question).query
            queries.append(query) # For inspection
            passages = self.retrieve(query).passages
            context_history.append(passages)
            context = deduplicate(context + passages)
    
        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(answer=pred.answer, queries=queries, context_history=context_history, question=question)

In [5]:

class ProgramWrapper:
    def __init__(self, name, program):
        self.name = name
        self.uncompiled = program()
        self.uncomplied_score = 0.0
        self.compiled = None
        self.compiled_score = 0.0


In [6]:
chatgpt = get_llm("openai", openai_key_path="~/.keys/openai_key.txt")

chroma_retriever = get_chroma_retriever("../chroma_db", "pdf_example")

In [7]:
chatgpt("what is BART model?")

['The BART model, which stands for Bayesian Additive Regression Trees, is a statistical model used for regression analysis. It combines the flexibility of regression trees with the Bayesian framework to provide a powerful tool for modeling complex relationships in data. BART models are particularly useful for handling high-dimensional data and capturing non-linear relationships between variables. They have been successfully applied in various fields such as finance, healthcare, and environmental science.']

In [8]:
dspy.settings.configure(lm=chatgpt, rm=chroma_retriever)

In [9]:
rag = MultiHopRAG()

In [10]:
rag("What kind of neural network is BART?")

Prediction(
    answer='BART is a type of neural network known as a sequence-to-sequence model.',
    queries=['"BART neural network architecture"', '"BART neural network architecture and design"'],
    context_history=[['Xu Jiang, Karl Cobbe, Tyna Eloundou, Gretchen\nKrueger, Kevin Button, Matthew Knight, Benjamin\nChess, and John Schulman. 2022. Webgpt: Browser-\nassisted question-answering with human feedback.\nLong Ouyang, Jeffrey Wu, Xu Jiang, Diogo Almeida,\nCarroll Wainwright, Pamela Mishkin, Chong Zhang,\nSandhini Agarwal, Katarina Slama, Alex Ray, et al.\n2022. Training language models to follow instruc-\ntions with human feedback. Advances in Neural\nInformation Processing Systems , 35:27730–27744.\nAaron Parisi, Yao Zhao, and Noah Fiedel. 2022. Talm:\nTool augmented language models.\nJohn V Pavlik. 2023. Collaborating with chatgpt: Con-\nsidering the implications of generative artificial intel-\nligence for journalism and media education. Journal-\nism & Mass Communication E

In [16]:
chroma_retriever("Architecture of BART neural network for natural language processing", 10)

[{'long_text': 'Xu Jiang, Karl Cobbe, Tyna Eloundou, Gretchen\nKrueger, Kevin Button, Matthew Knight, Benjamin\nChess, and John Schulman. 2022. Webgpt: Browser-\nassisted question-answering with human feedback.\nLong Ouyang, Jeffrey Wu, Xu Jiang, Diogo Almeida,\nCarroll Wainwright, Pamela Mishkin, Chong Zhang,\nSandhini Agarwal, Katarina Slama, Alex Ray, et al.\n2022. Training language models to follow instruc-\ntions with human feedback. Advances in Neural\nInformation Processing Systems , 35:27730–27744.\nAaron Parisi, Yao Zhao, and Noah Fiedel. 2022. Talm:\nTool augmented language models.\nJohn V Pavlik. 2023. Collaborating with chatgpt: Con-\nsidering the implications of generative artificial intel-\nligence for journalism and media education. Journal-\nism & Mass Communication Educator , 78(1):84–93.\nGabriel Poesia, Alex Polozov, Vu Le, Ashish Tiwari,\nGustavo Soares, Christopher Meek, and Sumit Gul-\nwani. 2022. Synchromesh: Reliable code generation\nfrom pre-trained language mo

In [13]:
chatgpt.inspect_history(n=3)





Write a search query that will help answer a complex question.

---

Follow the following format.

Context: Contexts produced from previous search queries.

Question: The complex question we began with.

Reasoning: Let's think step by step in order to ${produce the query}. We ...

Query: A search query that will help answer the question.

---

Context:
[1] «Xu Jiang, Karl Cobbe, Tyna Eloundou, Gretchen
Krueger, Kevin Button, Matthew Knight, Benjamin
Chess, and John Schulman. 2022. Webgpt: Browser-
assisted question-answering with human feedback.
Long Ouyang, Jeffrey Wu, Xu Jiang, Diogo Almeida,
Carroll Wainwright, Pamela Mishkin, Chong Zhang,
Sandhini Agarwal, Katarina Slama, Alex Ray, et al.
2022. Training language models to follow instruc-
tions with human feedback. Advances in Neural
Information Processing Systems , 35:27730–27744.
Aaron Parisi, Yao Zhao, and Noah Fiedel. 2022. Talm:
Tool augmented language models.
John V Pavlik. 2023. Collaborating with chatgpt: Con-
sidering t